## CP Recommendation System


In [ ]:
#importing stuff
import requests
import json
from pandas import json_normalize

#load data
tag_set = set()
data = requests.get("https://codeforces.com/api/problemset.problems").json()
problems = data["result"]["problems"]

df = json_normalize(problems)
print(df)

Now, lets find the different tags that are available in codeforces

In [3]:
# getting the tags
for problem in problems:
    if "tags" in problem:
        for tag in problem["tags"]:
            if not tag in tag_set:
                tag_set.add(tag)

# for printing the tags
with open("output.txt", "w") as out:
    for tag in tag_set:
        print(tag, file=out)

Now, its time to get the data from the users

In [50]:
#import csv

user_handle = input()
start, count = map(int, input().split())

user_url = ("https://codeforces.com/api/user.status?handle={}&from={}&count={}")
user_url = user_url.format(user_handle, start, count)
user_data = requests.get(user_url).json()

submissions = user_data["result"]

df2 = json_normalize(submissions)

#write_file = 'D:\Recommender'
#with open (write_file,'w', newline='') as fout:
#    writer = csv.writer(fout, linetermintor='\n')
#    writer.writerows(submission_list)

counter = 0
for item1, item2 in zip(list(df2.verdict), list(df2.id)):
    if item1 == 'OK':
        print(item2)

128248731
111040832
111003776
91798489
91773138
